In [1]:
import torch
from torch import nn
import seaborn as sns
import matplotlib.pyplot as plt
import pyro.distributions as dist
import pyro.distributions.transforms as T
import numpy as np
import torch.nn.functional as F

d=1
data = np.load('../../datasets/Combined_GWTC_m1m2chieffz.npz')
# m1 = torch.from_numpy(data["m1"]).float()[:, :1]

m1 = torch.from_numpy(data['m1'][:, :1])

def softplus_inv(y):
    return y + y.neg().expm1().neg().log()



dataset = m1.float()

# dataset = m1.log()
# dataset = m1

# dataset = softplus_inv(m1)

# dataset = (m1 - np.mean(m1)) / m1.std()

# d = 1

# dataset = torch.cat([torch.randn(64, d), torch.randn(8, d) + 8]).numpy()

# dataset = torch.from_numpy(dataset).float()

# dataset = torch.cat([5 + torch.randn(64, d).exp(), 80 + torch.randn(8, d)])

import numpy as np

# dataset = dataset.numpy()
# # dataset = (dataset - np.median(dataset)) / dataset.std()
# dataset = (dataset - dataset.mean()) / dataset.std()

# loc = softplus_inv(dataset).mean()
# scale = softplus_inv(dataset).std()

loc = dataset.mean()
scale = dataset.std()

dataset = (dataset - loc) / scale

sns.kdeplot(dataset.squeeze())

transforms = nn.ModuleList([T.planar(d) for _ in range(4)])
base_dist = dist.Normal(0, 1)

def log_prob(x, transforms, base_dist):
    J = 0
    for t in transforms:
        y = t(x)
        J += t.log_abs_det_jacobian(x, y)
        x = y
    log_prob = base_dist.log_prob(x).squeeze() + J
    return log_prob

steps = 32768
optimizer = torch.optim.Adam(transforms.parameters(), lr=1e-2)
# optimizer = torch.optim.RMSprop(transform_modules.parameters(), lr=5e-3, momentum=0.1)
for step in range(steps+1):
    optimizer.zero_grad()
    loss = -log_prob(dataset.clone(), transforms, base_dist).mean()
#     loss = -log_prob(flow_dist, dataset).mean()
    loss.backward()
    optimizer.step()
    
    
    
    if step % 500 == 0:
        print('step: {}, loss: {}'.format(step, loss.item()))

#         sample = flow_dist.sample((1000,)).squeeze().numpy()
#         sns.kdeplot(sample)
#         plt.title(f"Mean: {sample.mean()}, STD: {sample.std()}")
#         plt.show()
        
#         x = torch.linspace(-5, 5,128)
#         plt.plot(x, flow_dist.log_prob(x[:, None]).exp().detach().numpy().squeeze())
#         plt.show()

with torch.no_grad():
    x = dataset.clone()
    for t in transforms:
        x = t(x)

    sns.kdeplot(x.squeeze())

with torch.no_grad():
    x = torch.linspace(-5, 2, 128)[:, None]
    p = log_prob(x, transforms, base_dist).exp()
    
    plt.plot(x.squeeze().exp(), p.squeeze())

In [ ]:
sns.kdeplot(dataset.exp().squeeze())

In [ ]:
with torch.no_grad():
    sample = flow_dist.sample((1000,)).squeeze()
    sns.kdeplot(sample)
    plt.title(f"Mean: {sample.mean()}, STD: {sample.std()}")

In [ ]:
sns.kdeplot(sample.exp())

In [ ]:
x = torch.linspace(1, 120,128)

plt.plot(x, flow_dist.log_prob(x[:, None]).exp().detach().numpy().squeeze())

In [ ]:
sns.kdeplot(torch.exp(sample * dataset.std() + dataset.mean()))

In [ ]:
x = torch.linspace(-3, 2.5, 1024)

In [ ]:
plt.figure(figsize=(16, 16))


plt.plot(torch.exp(x * dataset.std() + dataset.mean()), flow_dist.log_prob(x[:, None]).exp().detach().numpy().squeeze())